In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # or any level higher than 0
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.model_selection import train_test_split

In [2]:
def load_images_from_directory(directory, label, num_images):
    images = []
    labels = []
    count = 0
    for file in os.listdir(directory):
        if count >= num_images:
            break
        img_path = os.path.join(directory, file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB format
        images.append(img)
        labels.append(label)
        count += 1
    return np.array(images), np.array(labels)

In [3]:
#Load training and testing datasets
train_real_dir = '/Users/nithinreddynagapur/Downloads/archive/real-vs-fake/train/real'
train_fake_dir = '/Users/nithinreddynagapur/Downloads/archive/real-vs-fake/train/fake'
test_real_dir = '/Users/nithinreddynagapur/Downloads/archive/real-vs-fake/test/real'
test_fake_dir = '/Users/nithinreddynagapur/Downloads/archive/real-vs-fake/test/fake'

In [4]:
num_images = 150
real_train, real_train_labels = load_images_from_directory(train_real_dir, 0, num_images)
fake_train, fake_train_labels = load_images_from_directory(train_fake_dir, 1, num_images)

real_test, real_test_labels = load_images_from_directory(test_real_dir, 0, num_images)
fake_test, fake_test_labels = load_images_from_directory(test_fake_dir, 1, num_images)

real_valid, real_test_valid = load_images_from_directory(valid_real_dir, 0, num_images)
fake_valid, fake_test_valid = load_images_from_directory(valid_fake_dir, 1, num_images)

# Combine real and fake images to form the final training and testing sets
X_train = np.concatenate((real_train, fake_train), axis=0)
y_train = np.concatenate((real_train_labels, fake_train_labels), axis=0)

X_test = np.concatenate((real_test, fake_test), axis=0)
y_test = np.concatenate((real_test_labels, fake_test_labels), axis=0)

X_valid = np.concatenate((real_valid, fake_valid), axis=0)
y_valid = np.concatenate((real_test_valid, fake_test_valid), axis=0)

# Shuffle the data to ensure a random distribution of real and fake images
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [11]:
X_train.shape

(300, 256, 256, 3)

In [12]:
X_train.shape[1], X_train.shape[2], X_train.shape[3]

(256, 256, 3)

In [31]:
def create_cnn(input_shape):
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(1, activation='softmax'))

    return model

In [32]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

In [33]:
model = create_cnn(input_shape)

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

  # Set your desired learning rate here
optimizer = Adam(learning_rate = 0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping loss
early_stopping_loss = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')

# Define early stopping accuracy
early_stopping_accuracy = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1, mode='max')

# Define model checkpointing
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Use the generator to augment the data during training
history = model.fit(X_train, y_train,
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_loss, early_stopping_accuracy, model_checkpoint])

In [41]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

test_data = X_test
test_labels = y_test

# Load the best model
best_model = load_model('best_model.h5')

# Make predictions on the test set
y_pred = best_model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate the evaluation metrics
accuracy = accuracy_score(test_labels, y_pred_classes)
precision = precision_score(test_labels, y_pred_classes, average='weighted', zero_division=0)
recall = recall_score(test_labels, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

# Print the classification report
print("\nClassification Report:")
print(classification_report(test_labels, y_pred_classes, zero_division=0))

10/10 [==============================] - 2s 169ms/step
Accuracy: 0.50
Precision: 0.25
Recall: 0.50

Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       150
           1       0.00      0.00      0.00       150

    accuracy                           0.50       300
   macro avg       0.25      0.50      0.33       300
weighted avg       0.25      0.50      0.33       300



In [42]:
from sklearn.linear_model import LogisticRegression

# Flatten the input images
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Create a logistic regression model
lr = LogisticRegression(max_iter = 1000)

# Train the model on the training set
lr.fit(X_train, y_train)

# Evaluate the model on the test set
score = lr.score(X_test, y_test)
print("Test accuracy:", score)

Test accuracy: 0.7366666666666667


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Reshape test data to 2D array
X_test = X_test.reshape(X_test.shape[0], -1)

# Create logistic regression model
lr_model = LogisticRegression(max_iter = 1000)

# Train logistic regression model on training data
lr_model.fit(X_train, y_train)

# Make predictions on test data
y_pred = lr_model.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

# Print the classification report
print("\nClassification Report:")
print(classification_report(test_labels, y_pred, zero_division=0))

Accuracy: 0.7366666666666667
Precision: 0.7472259622080877
Recall: 0.7366666666666667
F1 score: 0.7338245038690909

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.63      0.71       150
           1       0.70      0.84      0.76       150

    accuracy                           0.74       300
   macro avg       0.75      0.74      0.73       300
weighted avg       0.75      0.74      0.73       300



In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



# Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# Evaluate RF Classifier
y_pred_rf = rf_clf.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_precision = precision_score(y_test, y_pred_rf, average='macro')
rf_recall = recall_score(y_test, y_pred_rf, average='macro')
rf_f1 = f1_score(y_test, y_pred_rf, average='macro')
print("Random Forest Classifier Accuracy: {:.4f}".format(rf_acc))
print("Random Forest Classifier Precision: {:.4f}".format(rf_precision))
print("Random Forest Classifier Recall: {:.4f}".format(rf_recall))
print("Random Forest Classifier F1 Score: {:.4f}".format(rf_f1))

# Print the classification report
print("\nClassification Report:")
print(classification_report(test_labels, y_pred, zero_division=0))

Random Forest Classifier Accuracy: 0.6667
Random Forest Classifier Precision: 0.6668
Random Forest Classifier Recall: 0.6667
Random Forest Classifier F1 Score: 0.6666

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.63      0.71       150
           1       0.70      0.84      0.76       150

    accuracy                           0.74       300
   macro avg       0.75      0.74      0.73       300
weighted avg       0.75      0.74      0.73       300



In [45]:
# Support Vector Classifier
svc_clf = SVC(kernel='rbf', random_state=42)
svc_clf.fit(X_train, y_train)

# Evaluate SVC Classifier
y_pred_svc = svc_clf.predict(X_test)
svc_acc = accuracy_score(y_test, y_pred_svc)
svc_precision = precision_score(y_test, y_pred_svc, average='macro')
svc_recall = recall_score(y_test, y_pred_svc, average='macro')
svc_f1 = f1_score(y_test, y_pred_svc, average='macro')
print("Support Vector Classifier Accuracy: {:.4f}".format(svc_acc))
print("Support Vector Classifier Precision: {:.4f}".format(svc_precision))
print("Support Vector Classifier Recall: {:.4f}".format(svc_recall))
print("Support Vector Classifier F1 Score: {:.4f}".format(svc_f1))

# Print the classification report
print("\nClassification Report:")
print(classification_report(test_labels, y_pred, zero_division=0))

Support Vector Classifier Accuracy: 0.6700
Support Vector Classifier Precision: 0.6700
Support Vector Classifier Recall: 0.6700
Support Vector Classifier F1 Score: 0.6700

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.63      0.71       150
           1       0.70      0.84      0.76       150

    accuracy                           0.74       300
   macro avg       0.75      0.74      0.73       300
weighted avg       0.75      0.74      0.73       300

